# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark

In [3]:
findspark.init("/home/ratzz/spark-2.4.4-bin-hadoop2.7") #Using Virtual Box

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("project").getOrCreate() #spark instance

In [16]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [7]:
df.printSchema() #Schema

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
for items in df.head(2)[0]:
    print(items)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [9]:
#all functions import here
from pyspark.sql.functions import (countDistinct,format_number,)

# EDA

In [10]:
#20 distinct cruise line 
df.select(countDistinct("Cruise_line")).show()

+---------------------------+
|count(DISTINCT Cruise_line)|
+---------------------------+
|                         20|
+---------------------------+



In [11]:
#Describe
for items in df.describe().head(10):
    print(items)
    print("\n")

Row(summary='count', Ship_name='158', Cruise_line='158', Age='158', Tonnage='158', passengers='158', length='158', cabins='158', passenger_density='158', crew='158')


Row(summary='mean', Ship_name='Infinity', Cruise_line=None, Age='15.689873417721518', Tonnage='71.28467088607599', passengers='18.45740506329114', length='8.130632911392404', cabins='8.830000000000005', passenger_density='39.90094936708861', crew='7.794177215189873')


Row(summary='stddev', Ship_name='NaN', Cruise_line=None, Age='7.615691058751413', Tonnage='37.229540025907866', passengers='9.677094775143416', length='1.793473548054825', cabins='4.4714172221480615', passenger_density='8.63921711391542', crew='3.503486564627034')


Row(summary='min', Ship_name='Adventure', Cruise_line='Azamara', Age='4', Tonnage='2.329', passengers='0.66', length='2.79', cabins='0.33', passenger_density='17.7', crew='0.59')


Row(summary='max', Ship_name='Zuiderdam', Cruise_line='Windstar', Age='48', Tonnage='220.0', passengers='54.0', le

In [34]:
# df.show()

In [12]:
group_df = df.groupBy("Cruise_line").mean()

In [13]:
# group_df.show()

In [14]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [15]:
# df = df.select(['Cruise_line',
#  'Age',
#  'Tonnage',
#  'passengers',
#  'length',
#  'cabins',
#  'passenger_density',"crew"])

In [103]:
# training_Data.show()

+-----------+---+------------------+----------+------+------+-----------------+
|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|
+-----------+---+------------------+----------+------+------+-----------------+
|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|
|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|
|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8|
|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|
|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|
|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29|
|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29|
|   Carnival| 23|            70.367|     20.56|  8.55| 10.22|            34.23|
|   Carnival| 19|            70.367|     20.52|  8.55|  10.2|            34.29|
|   Carnival|  6|110.23899999999999|    

In [104]:
# training_Data.printSchema()

root
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)



In [17]:
#Using String Indexer to convert Cruise line to number type
from pyspark.ml.feature import StringIndexer

In [18]:
indexer = StringIndexer(inputCol="Cruise_line",outputCol="Cruise_lineIndexed")

In [19]:
indexed = indexer.fit(df).transform(df)

In [20]:
indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndexed: double (nullable = false)



In [21]:
# from pyspark.sql.types import DoubleType
# Train_final=indexed.select(['Cruise_lineIndexed',
#  'Age',
#  'passengers',
#  'length',
#  'cabins',
#  'passenger_density',"Tonnage"])

In [22]:
# Train_final.printSchema()

In [23]:
from pyspark.ml.feature import OneHotEncoderEstimator

In [24]:
one_hot= OneHotEncoderEstimator(inputCols=['Cruise_lineIndexed'],outputCols=["Cruise_line_vect"])

In [25]:
one_hot_encoded = one_hot.fit(indexed).transform(indexed)

In [26]:
one_hot_encoded.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_lineIndexed',
 'Cruise_line_vect']

In [116]:
# features_Data = one_hot_encoded.select(['Age',
#  'passengers',
#  'length',
#  'cabins',
#  'passenger_density',
#  'Tonnage',
#  'Cruise_line_vect'])

In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [28]:
assembler = VectorAssembler(inputCols=['Age',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Tonnage',
 'Cruise_line_vect'],outputCol="features")

In [29]:
output = assembler.transform(one_hot_encoded)

In [30]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndexed: double (nullable = false)
 |-- Cruise_line_vect: vector (nullable = true)
 |-- features: vector (nullable = true)



In [32]:
train_data, test_data = output.randomSplit([0.7,0.3])

In [35]:
train_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_lineIndexed',
 'Cruise_line_vect',
 'features']

In [36]:
from pyspark.ml.regression import LinearRegression

In [37]:
lr = LinearRegression(labelCol="crew")

In [38]:
lrModel = lr.fit(train_data)

In [39]:
test_value = lrModel.evaluate(test_data)

In [40]:
test_value.rootMeanSquaredError

0.6050283792910605

In [41]:
test_value.r2

0.9726994714588775

In [54]:
test_data.describe().show()

+-------+---------+-----------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+
|summary|Ship_name|Cruise_line|               Age|          Tonnage|        passengers|           length|            cabins|passenger_density|             crew|Cruise_lineIndexed|
+-------+---------+-----------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+
|  count|       35|         35|                35|               35|                35|               35|                35|               35|               35|                35|
|   mean|     null|       null|16.914285714285715|70.40559999999998|18.565428571428573|7.913428571428573| 8.642000000000001|39.90085714285714|7.437999999999998| 5.142857142857143|
| stddev|     null|       null| 8.905951178091795|42.57213808984905| 11.65006471652238|2.05266676310

In [42]:
test_value.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -0.822217850501007|
| -0.3570051891198709|
|-0.18840907734201195|
|  0.8640332059106992|
|  1.8166889212225383|
|-0.15096328080813137|
|  0.5850076461615128|
| 0.13162924653934915|
| -0.3348416011358353|
| 0.13430090934850014|
|-0.15677532014122697|
| -0.1507217162032113|
|  0.0723980902770549|
|-0.00732767722882...|
|-0.09158869097060363|
| -0.4257382311723781|
|  0.9508815647274638|
| 0.11412001234177538|
|  -0.425496666567458|
|  0.5062060070090908|
+--------------------+
only showing top 20 rows



In [45]:
unlabled_data = test_data.select("features")

In [47]:
unlabled_data.show() #lets deploy some unlabled features 

+--------------------+
|            features|
+--------------------+
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5]...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
|(25,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [48]:
predictions = lrModel.transform(unlabled_data)

In [49]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(25,[0,1,2,3,4,5,...| 1.412217850501007|
|(25,[0,1,2,3,4,5,...|11.357005189119871|
|(25,[0,1,2,3,4,5,...| 9.188409077342012|
|(25,[0,1,2,3,4,5,...|11.515966794089302|
|(25,[0,1,2,3,4,5,...| 4.893311078777462|
|(25,[0,1,2,3,4,5,...| 7.750963280808131|
|(25,[0,1,2,3,4,5,...|0.8749923538384872|
|(25,[0,1,2,3,4,5,...|  9.06837075346065|
|(25,[0,1,2,3,4,5,...|11.834841601135835|
|(25,[0,1,2,3,4,5,...|  13.4656990906515|
|(25,[0,1,2,3,4,5,...|11.256775320141227|
|(25,[0,1,2,3,4,5,...| 7.750721716203211|
|(25,[0,1,2,3,4,5,...|13.527601909722945|
|(25,[0,1,2,3,4,5,...|  8.22732767722883|
|(25,[0,1,2,3,4,5]...|3.5915886909706036|
|(25,[0,1,2,3,4,5,...|12.275738231172378|
|(25,[0,1,2,3,4,5,...| 4.399118435272536|
|(25,[0,1,2,3,4,5,...| 9.875879987658225|
|(25,[0,1,2,3,4,5,...|12.275496666567458|
|(25,[0,1,2,3,4,5,...|  8.49379399299091|
+--------------------+------------

In [52]:
# df.filter("crew < 2").show()

# Made By Ratanakar Maurya